In [6]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("data/student.csv")
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [3]:
X = df.drop("math_score", axis=1)
y = df["math_score"]

In [4]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features)
    ]
)

In [5]:
X = preprocessor.fit_transform(X)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

In [11]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2score = r2_score(true, predicted)
    return mae, mse, rmse, r2score

In [13]:
from src.components import model_trainer


models = {
    "Linear Regression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "K-Neighbors Regressor" : KNeighborsRegressor(),
    "Decision Tree" : DecisionTreeRegressor(),
    "Random Forest Regressor" : RandomForestRegressor(),
    "XGB Regressor" : XGBRegressor(),
    "CatBoost Regressor" : CatBoostRegressor(verbose=False),
    "AdaBoost Regressor" : AdaBoostRegressor()
}
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae, model_train_mse, model_train_rmse, model_train_r2_score = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2_score = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Model Performance for training set")
    print("- Root mean squared error: {:.4f}".format(model_train_rmse))
    print("- Mean absolute error: {:.4f}".format(model_train_mae))
    print("- R2 Square: {:.4f}".format(model_train_r2_score))

    print("-"*25)

    print("Model Performance for test set")
    print("- Root mean squared error: {:.4f}".format(model_test_rmse))
    print("- Mean absolute error: {:.4f}".format(model_test_mae))
    print("- R2 Square: {:.4f}".format(model_test_r2_score))    
    r2_list.append(model_test_r2_score)

    print("="*25)
    print("\n")

Linear Regression
Model Performance for training set
- Root mean squared error: 5.3231
- Mean absolute error: 4.2667
- R2 Square: 0.8743
-------------------------
Model Performance for test set
- Root mean squared error: 5.3940
- Mean absolute error: 4.2148
- R2 Square: 0.8804


Lasso
Model Performance for training set
- Root mean squared error: 6.5938
- Mean absolute error: 5.2063
- R2 Square: 0.8071
-------------------------
Model Performance for test set
- Root mean squared error: 6.5197
- Mean absolute error: 5.1579
- R2 Square: 0.8253


Ridge
Model Performance for training set
- Root mean squared error: 5.3233
- Mean absolute error: 4.2650
- R2 Square: 0.8743
-------------------------
Model Performance for test set
- Root mean squared error: 5.3904
- Mean absolute error: 4.2111
- R2 Square: 0.8806


K-Neighbors Regressor
Model Performance for training set
- Root mean squared error: 5.7172
- Mean absolute error: 4.5270
- R2 Square: 0.8550
-------------------------
Model Performance